In [1]:
from netCDF4 import Dataset as ds
import numpy as np
from e582utils.data_read import download
import datetime
import dateutil as du 
import parser
import pytz
import dateutil.parser 
import pandas as pd

In [ ]:
# Function to slice the filename with .nc extention, this will be used to open the files in directory
def file_info(ncfilename):
    with ds(ncfilename,'r') as ncdat:
        array = xr.open_dataset(ncfilename) # open_dataset works fine with either full path or filename.nc 
        return array  
def array_sel(array):
    dsloc = array.sel(lon=lon,lat=lat,method='nearest') 
    val_arr = dsloc['analysed_sst']
    return val_arr 

# Functions

In [8]:
# This function would provide the array of par, lat and lon of the file, with this data 
# I am going to get "the_par" 
def sst_lat_lon(infile): 
    with ds(infile,'r') as ncfile:
        ncfile.set_auto_mask(False)
        sst=ncfile.variables['analysed_sst'][...]
        lat=ncfile.variables['lat'][...]
        lon=ncfile.variables['lon'][...]
        fill_value=ncfile.variables['analysed_sst']._FillValue
        hit = sst == fill_value
        sst[hit] = np.nan 
        #sat=getattr(ncfile,'platform')
    return sst,lat,lon #,sat   

# Global function 

In [11]:
import glob, os
from os.path import expanduser
from pathlib import Path
home = Path(expanduser("~"))
datapath = home / Path ('E:/NetCDF/SST_CRW') #('C:/Users/bmped/Documents/UBC/Classes/EOSC_582/project/SST')#()
all_files=list(datapath.glob('*nc')) #Return a list of path names that match pathname, in this case all the .nc´s
data_list=[]
for infile in all_files:
    file_info=file_info(infile)# this line should gave this: datetime.datetime(2017, 3, 12, 0, 0, tzinfo=<UTC>)
    sst,lat,lon=sst_lat_lon(infile) # this would provide the array of "lat-lon" that are going to be used to get the_par values of each site 
#     for site_name,site_dict in sites_short.items(): 
#         date_dict=date_info(the_date)# this would give the year: 2017 e.g.
# #         print(site_name)
#         site_lat,site_lon=site_dict['lat'],site_dict['lon']
#         lat_idx = np.abs(lat-site_lat).argmin()
#         lon_idx = np.abs(lon-site_lon).argmin()
#         # type(sst) 
#         sst=sst.squeeze() # Here we used ".squeeze" to tackle the multilevel issue. In this case we have 4 levels. 
#         # sst.shape   # This is to check if we are in the correct level. 
#         the_sst = sst[lat_idx,lon_idx]      
#         date_dict['analysed_sst']=the_sst
#         date_dict['site_name']=site_name
#         #date_dict['sat']=sat
#         #print(site_name,the_par)
#         data_list.append(date_dict)
                     


In [1]:
# df=pd.DataFrame.from_records(data_list)
# df.columns
# # sst_vals=df['analysed_sst'].values
# # hit=sst_vals < 0
# # sst_vals[hit]=np.nan
# # df['sst']=sst_vals
# df


In [13]:
# BREAK
with pd.HDFStore('df_SST_CRW.h5','w') as outfile:  # Storage the dataframe 
    outfile.put('df_SST_CRW',df,format = 'table')